In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,city_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Bredasdorp,ZA,2022-04-03 15:24:58,-34.5322,20.0403,67.44,70,71,12.82
1,1,Castro,BR,2022-04-03 15:21:47,-24.7911,-50.0119,66.09,85,100,9.24
2,2,Butaritari,KI,2022-04-03 15:24:59,3.0707,172.7902,81.21,71,65,14.83
3,3,Hobart,AU,2022-04-03 15:24:59,-42.8794,147.3294,57.06,93,100,4.61
4,4,Upernavik,GL,2022-04-03 15:24:59,72.7868,-56.1549,16.61,81,79,8.81


In [3]:
city_data_df.dtypes

city_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat","Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations,weights=[max(temp, 0) for temp in max_temp], 
                                 dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind_speed = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind_speed, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [13]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,city_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Butaritari,KI,2022-04-03 15:24:59,3.0707,172.7902,81.21,71,65,14.83
6,6,Bata,GQ,2022-04-03 15:25:00,1.8639,9.7658,84.06,72,36,10.29
11,11,Morant Bay,JM,2022-04-03 15:25:02,17.8815,-76.4093,87.71,67,10,9.17
18,18,Foumbot,CM,2022-04-03 15:25:05,5.5079,10.6356,89.76,26,100,4.21
29,29,Dingle,PH,2022-04-03 15:25:10,10.9995,122.6711,76.57,85,51,6.69
34,34,Port-Gentil,GA,2022-04-03 15:25:12,-0.7193,8.7815,85.96,70,20,13.80
35,35,Rikitea,PF,2022-04-03 15:25:12,-23.1203,-134.9692,75.87,64,41,8.79
37,37,Lazaro Cardenas,MX,2022-04-03 15:25:13,17.9583,-102.2000,76.10,60,1,0.34
38,38,Jiwani,PK,2022-04-03 15:25:14,25.0500,61.7417,80.13,62,21,9.06
43,43,Bambous Virieux,MU,2022-04-03 15:25:16,-20.3428,57.7575,79.21,78,75,12.66


In [14]:
preferred_cities_df.count()

city_ID       169
City          169
Country       169
Date          169
Lat           169
Lng           169
Max Temp      169
Humidity      169
Cloudiness    169
Wind Speed    169
dtype: int64

In [15]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Butaritari,KI,81.21,3.0707,172.7902,
6,Bata,GQ,84.06,1.8639,9.7658,
11,Morant Bay,JM,87.71,17.8815,-76.4093,
18,Foumbot,CM,89.76,5.5079,10.6356,
29,Dingle,PH,76.57,10.9995,122.6711,
34,Port-Gentil,GA,85.96,-0.7193,8.7815,
35,Rikitea,PF,75.87,-23.1203,-134.9692,
37,Lazaro Cardenas,MX,76.10,17.9583,-102.2000,
38,Jiwani,PK,80.13,25.0500,61.7417,
43,Bambous Virieux,MU,79.21,-20.3428,57.7575,


In [24]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the locations
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
    info_box_template = """
    <dl>
    <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
    <dt>City</dt><dd>{City}</dd>
    <dt>Country</dt><dd>{Country}</dd>
    <dt>Max Temp</dt><dd>{Max Temp} °F</dd>
    </dl>
    """
    
    # Store the DataFrame Row.
    hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [28]:
hotel_df.head()

['\n    <dl>\n    <dt>Hotel Name</dt><dd>Isles Sunset Lodge</dd>\n    <dt>City</dt><dd>Butaritari</dd>\n    <dt>Country</dt><dd>KI</dd>\n    <dt>Max Temp</dt><dd>81.21 °F</dd>\n    </dl>\n    ',
 '\n    <dl>\n    <dt>Hotel Name</dt><dd>Hotel Elik Melen</dd>\n    <dt>City</dt><dd>Bata</dd>\n    <dt>Country</dt><dd>GQ</dd>\n    <dt>Max Temp</dt><dd>84.06 °F</dd>\n    </dl>\n    ',
 '\n    <dl>\n    <dt>Hotel Name</dt><dd>Whispering Bamboo Cove Resort</dd>\n    <dt>City</dt><dd>Morant Bay</dd>\n    <dt>Country</dt><dd>JM</dd>\n    <dt>Max Temp</dt><dd>87.71 °F</dd>\n    </dl>\n    ',
 '\n    <dl>\n    <dt>Hotel Name</dt><dd>HÔTEL ROYAL</dd>\n    <dt>City</dt><dd>Foumbot</dd>\n    <dt>Country</dt><dd>CM</dd>\n    <dt>Max Temp</dt><dd>89.76 °F</dd>\n    </dl>\n    ',
 '\n    <dl>\n    <dt>Hotel Name</dt><dd>SEGAYA PROPERTY</dd>\n    <dt>City</dt><dd>Dingle</dd>\n    <dt>Country</dt><dd>PH</dd>\n    <dt>Max Temp</dt><dd>76.57 °F</dd>\n    </dl>\n    ',
 '\n    <dl>\n    <dt>Hotel Name</dt><d

In [29]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations,  info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))